In [6]:
from lxml import etree
import sqlite3

In [7]:
xml_db = 'wordnet/data/xml_entries.db'
sset_db = 'wordnet/data/all_synsets.db'
xml_rel_db = 'wordnet/data/xml_relations.db'

In [8]:
tree = etree.parse("estwn-et-2.0.0.beta.xml")

In [9]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print("Connection error: [%s]" % e)

    return None

def create_table(conn, create_table_sql ):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except sqlite3.Error as e:
        print("Connection error while creating table: [%s]" % e)


def entryTable(databaseLoc):

    sql_create_synset_table = ''' CREATE TABLE IF NOT EXISTS xml_LexicalEntry(
                                        
                                        estwn_id TEXT NOT NULL,
                                        writtenForm TEXT NOT NULL,
                                        pos TEXT NOT NULL,
                                        sense INT NOT NULL,
                                        status TEXT NOT NULL
                                                    ); '''
    conn = create_connection(databaseLoc)
    if conn is not None:
        create_table(conn,sql_create_synset_table)
    else:
        print("Error! cannot create db conn.")        

In [10]:

def fetch_entries(tree):
    synsets = tree.xpath('/LexicalResource/Lexicon/LexicalEntry')
    j=0
    for sset in synsets:
        if len(sset)>1:
            for i in range(1, len(sset)):
                status_list.append(sset[i].attrib['status'])
                sset_list.append(sset[i].attrib['id'])
                pos_list.append(sset[0].attrib['partOfSpeech'])
                lemma_list.append(sset[0].attrib['writtenForm'])    
                sense = sset[i].attrib['id']
                sense_list.append(sense[-1:])
                
                
        else:
            try:
                pos_list.append(sset[0].attrib['partOfSpeech'])
                lemma_list.append(sset[0].attrib['writtenForm'])
                word = sset[0].attrib['writtenForm']
                
                if " " in word:
                    var = word.replace(" ", "_")
                    word = var
                sset_list.append( "s-" + word + "-" + sset[0].attrib['partOfSpeech'] )
                
                estwnId_list.append(sset.attrib['id'])
                sense = sset.attrib['id']
                sense_list.append(sense[-1:])
                status_list.append("None")
            except:
                print("pass @",j)
                pass
        j+=1
        '''
        for relations
        
        if len(sset) > 2:
            for data in sset:
                #print(data.attrib)
                
                status_list.append(data.attrib['target'])
                conf_list.append((data.attrib['confidenceScore']))
                source_sense.append(data[0].attrib['sourceSense'])           
                '''

In [11]:
def upload_entries(db_file):
    entryTable(db_file)
    conn = create_connection(db_file)
    cursor = conn.cursor()
    
    with conn:
        for i in range(len(sset_list)):
              
            cursor.execute("INSERT INTO XML_LexicalEntry(estwn_id, writtenForm, pos, sense, status) VALUES(?,?,?,?, ?)"\
                                                        ,(estwnId_list[i],lemma_list[i], pos_list[i], sense_list[i], status_list[i]))
            conn.commit()

In [12]:
def fetch_lemma(estwn_id):
    '''
    
    '''
    xml_conn = create_connection(xml_db)
    cursor = xml_conn.cursor()
    with xml_conn:
        #select all synsets with different indices from table.
        cursor.execute("SELECT DISTINCT writtenForm FROM XML_LexicalEntry WHERE estwn_id = ?", (estwn_id,))
        row = cursor.fetchone()
        #print(row[0])
        if row is not None:
            return row[0]

In [13]:
def fetch_ssetId(estwn_id):
    
    lemma = fetch_lemma(estwn_id)
    
    db_conn = create_connection(sset_db)
    cursor = db_conn.cursor()
    with db_conn:
        cursor.execute("SELECT DISTINCT synset_id FROM synset_table WHERE synset_word = ?", (lemma,))
        row = cursor.fetchone()
        if row is not None:
            #print(row[0])
            return row[0]

In [16]:

status_list=[]
pos_list = []
lemma_list = []
sset_list=[]
sense_list=[]


fetch_entries(tree)

In [90]:


upload_entries(xml_db)

133161
133161
133161
133161
133161
133161
